## 1. Set Up Environment

In [29]:
import pandas as pd
import requests
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

## 2. Expedia Scraper

In [35]:
flight_inputs = {'Roundtrip': 'ROUND_TRIP', 'One-way': 'ONE_WAY', 'Multi-city': 'MULTI_CITY', 
                 'class': ['Economy', 'Premium economy', 'Business class', 'First class']}

def expedia_flight(type_='Roundtrip', class_='Economy'):

    chrome_driver = "C:/ChromeDrive/chromedriver"
    browser = webdriver.Chrome(chrome_driver)
    url = "https://www.expedia.ca/"
    browser.get(url)
    time.sleep(0.2)
    
    # Select "Flights" tab
    flight_xpath = '//a[@aria-controls="search_form_product_selector_flights"]'
    flight_element = browser.find_element_by_xpath(flight_xpath)
    flight_element.click()
    time.sleep(0.2)
    
    # Select "type" of flight
    type_xpath = '//a[@aria-controls="FlightSearchForm_' + flight_inputs[type_] + '"]'
    type_element = browser.find_element_by_xpath(type_xpath)
    type_element.click()
    time.sleep(0.2)
    
    # Select ticket "class"
    class_xpath = '//span[@class="uitk-pill-text"]'
    class_element = browser.find_element_by_xpath(class_xpath)
    class_element.click()
    time.sleep(1)
    class_element.send_keys(Keys.ARROW_DOWN)

In [ ]:
expedia_flight('One-way')